In [80]:
import requests
# These two things are for Pandas, it widens the notebook and lets us display data easily.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


In [81]:
!pip install spotipy
#!pip3 install spotipy --user requests

In [82]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 
#To access authorised Spotify data
client_id = '693faf0bce4f4fb0aa5fc5d8b3cc0b25'
client_secret = 'c2ab38a172f44774a0cfae1d7734130f'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

#spotify object to access API
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 
#artist will have to provide their own ID that is assigned to them by spotify
#this is because small bands do not often pop up as the first search return so we cannot rely on that
#there are also duplicate names

#not deemed reliable yet
def get_artistID_by_artistName(artistName):
        artist = sp.search(q=artistName, type="artist")
        #print(artist)
        artistObject = artist['artists']['items']
        if len(artistObject) == 0:
            return "NOT IN SPOTIFY"
        else:
            artistID = artistObject[0]['id']
            #print('AID: ', artistID)
            return artistID;
    
def get_genre(artistID):
    #navigate to artist
    artist = sp.artist(artistID)
    #Obtain genre
    genres = artist['genres']
    if len(genres) == 0:
        return 'NO GENRES LISTED'
    else:
        return genres

def get_followers(artistID):
    #navigate to artist
    artist = sp.artist(artistID)
    #Obtain follower count
    followers = int(artist['followers']['total'])
    return followers

In [83]:
import requests

!pip install lxml
from lxml import html

In [84]:
import re

#Scrape artist about section to get top cities
#parse for top cities
def get_cities(artistID):
    r = requests.get('https://open.spotify.com/artist/{}/about'.format(artistID), timeout=10)
    match = re.search(r'Where People Listen', r.text)
    #get where people listen starting point
    string = r.text[match.start():]
    #split at colon
    strings = string.split(':')
    strings1 = strings[1].split('"')
    cities = strings1[0].split(",")
    #grab second string
    return cities

def get_monthlyListeners(artistID):
    r = requests.get('https://open.spotify.com/artist/{}/about'.format(artistID), timeout=10)
    match = re.search(r'Monthly Listeners', r.text)
    #get monthly listeners starting point
    string = r.text[match.start():]
    #split at colon# Set Up DataFrame with Band Names
import pandas as pd

#use names from class... THIS PART WILL BE CHANGED!

names = ["Turnstile", "NoMBe", "Shabaka Hutchings", "Hippo Campus", "Sunday Morning Detox", "Joji", "Jungle Giants", "Magic City Hippies", "Sudan Archives", "Lil Jodeci"]

df = pd.DataFrame({"band_name": names})

df


,band_name
0,Turnstile
1,NoMBe
2,Shabaka Hutchings
3,Hippo Campus
4,Sunday Morning Detox
5,Joji
6,Jungle Giants
7,Magic City Hippies
8,Sudan Archives
9,Lil Jodeci


In [85]:
df["id"] = df.band_name.apply(get_artistID_by_artistName)
df["genre"] = df.id.apply(get_genre)
df["top_locales"] = df.id.apply(get_cities)
df["listeners"] = df.id.apply(get_monthlyListeners)
df["followers"] = df.id.apply(get_followers)
pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.max_colwidth',None)
df

,band_name,id,genre,top_locales,listeners,followers
0,Turnstile,2qnpHrOzdmOo1S4ox3j17x,[hardcore punk],"[ Los Angeles, Chicago, London, Dallas, New York City]",None,101949
1,NoMBe,5Lhxlge1CR1DrgDAje8Qaw,"[electropop, indie poptimism, indie soul, modern alternative rock, modern rock, vapor soul]","[ Los Angeles, Chicago, Denver, Dallas, Sydney]",None,148220
2,Shabaka Hutchings,6ywMpa6AmGJpV5Sbyy58Js,[british jazz],"[ London, Amsterdam, Berlin, Bristol, Paris]",None,14242
3,Hippo Campus,1btWGBz4Uu1HozTwb2Lm8A,"[indie pop, indie poptimism, indie rock, minneapolis indie, modern alternative rock, modern rock]","[ Chicago, Los Angeles, Dallas, Atlanta, New York City]",None,407756
4,Sunday Morning Detox,0ebYhazPi5731pDNzITJ8y,NO GENRES LISTED,"[ New Orleans, Houston, Los Angeles, Dallas, Chicago]",None,99
5,Joji,3MZsBdqDrRTJihTHQrO6Dq,"[alternative r&b, viral pop]","[ Los Angeles, Chicago, Jakarta, Sydney, Dallas]",None,3575555
6,Jungle Giants,6wFwvxJkurQPU2UdeD4qVt,"[australian indie, australian pop]","[ Sydney, Brisbane, Melbourne, Perth, Adelaide]",None,183338
7,Magic City Hippies,1ikg4sypcURm8Vy5GP68xb,"[indie pop, indie rock, miami indie, modern rock, vapor soul]","[ Los Angeles, Chicago, Denver, Dallas, Atlanta]",None,81360
8,Sudan Archives,2MPHBxznH1fj59jbOWY38u,"[afrofuturism, alternative r&b, art pop, escape room, indie soul, la pop]","[ London, Los Angeles, Berlin, Brooklyn, Chicago]",None,115900
9,Lil Jodeci,29LX8E7dQwhUFl04odAznZ,NO GENRES LISTED,"[ Los Angeles, New Orleans, Denver, Dallas, Minneapolis]",None,66


In [86]:
import numpy as np

genre_map = {'funk':0,
            'rock' or 'punk':1,
            'reggae': 2,
            'r&b': 3,
            'rap': 4,
           'pop':5,
           'jazz':6,
           'electronic':7,
           'indie':8,
            'country':9}

In [87]:
df['mapped_genre'] = [[] for _ in range(df.shape[0])]

In [88]:
for i in range(df.genre.size):
    if any("funk" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('0')
        
    if any("rock" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('1')
        
    if any("punk" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('1')
        
    if any("reggae" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('2')
        
    if any("r&b" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('3')
        
    if any("rap" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('4')
        
    if any("pop" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('5')
        
    if any("jazz" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('6')
        
    if any("electronic" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('7')
        
    if any("indie" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('8')
        
    if any("country" in s for s in df.genre[i]):
        df['mapped_genre'][i].append('9')

In [89]:
df

,band_name,id,genre,top_locales,listeners,followers,mapped_genre
0,Turnstile,2qnpHrOzdmOo1S4ox3j17x,[hardcore punk],"[ Los Angeles, Chicago, London, Dallas, New York City]",None,101949,[1]
1,NoMBe,5Lhxlge1CR1DrgDAje8Qaw,"[electropop, indie poptimism, indie soul, modern alternative rock, modern rock, vapor soul]","[ Los Angeles, Chicago, Denver, Dallas, Sydney]",None,148220,"[1, 5, 8]"
2,Shabaka Hutchings,6ywMpa6AmGJpV5Sbyy58Js,[british jazz],"[ London, Amsterdam, Berlin, Bristol, Paris]",None,14242,[6]
3,Hippo Campus,1btWGBz4Uu1HozTwb2Lm8A,"[indie pop, indie poptimism, indie rock, minneapolis indie, modern alternative rock, modern rock]","[ Chicago, Los Angeles, Dallas, Atlanta, New York City]",None,407756,"[1, 5, 8]"
4,Sunday Morning Detox,0ebYhazPi5731pDNzITJ8y,NO GENRES LISTED,"[ New Orleans, Houston, Los Angeles, Dallas, Chicago]",None,99,[]
5,Joji,3MZsBdqDrRTJihTHQrO6Dq,"[alternative r&b, viral pop]","[ Los Angeles, Chicago, Jakarta, Sydney, Dallas]",None,3575555,"[3, 5]"
6,Jungle Giants,6wFwvxJkurQPU2UdeD4qVt,"[australian indie, australian pop]","[ Sydney, Brisbane, Melbourne, Perth, Adelaide]",None,183338,"[5, 8]"
7,Magic City Hippies,1ikg4sypcURm8Vy5GP68xb,"[indie pop, indie rock, miami indie, modern rock, vapor soul]","[ Los Angeles, Chicago, Denver, Dallas, Atlanta]",None,81360,"[1, 5, 8]"
8,Sudan Archives,2MPHBxznH1fj59jbOWY38u,"[afrofuturism, alternative r&b, art pop, escape room, indie soul, la pop]","[ London, Los Angeles, Berlin, Brooklyn, Chicago]",None,115900,"[3, 5, 8]"
9,Lil Jodeci,29LX8E7dQwhUFl04odAznZ,NO GENRES LISTED,"[ Los Angeles, New Orleans, Denver, Dallas, Minneapolis]",None,66,[]


In [90]:
df['percentage_genre'] = [[] for _ in range(df.shape[0])]

In [91]:
import requests

In [110]:
# Define API Key
api_key = "FYSeGSNlg6HXIfAr"

# Define Metro ID Key
metro_id = "11772"

# request json
r = requests.get("https://api.songkick.com/api/3.0/metro_areas/"+ metro_id +"/calendar.json?apikey=" + api_key)

# parse for events
events = r.json()["resultsPage"]["results"]["event"]

# define dictionary for storage
venue_artist_dict = {}

#print(get_artistID_by_artistName('DOJO (US)'))

#loop through events
for i in range(len(events)-1):
    
    # The artist
    act = events[i+1]["performance"][0]["artist"]["displayName"]
    #print(act)
    
    # Venue Name
    venue = events[i+1]["venue"]["displayName"]
    
    # if the venue is not already in dict, add it with the artist as its value
    if venue not in venue_artist_dict.keys():
        #print(act)
        #if act == 'NOT IN SPOTIFY':
        #venue_genre = 'NONE'
        id_return = get_artistID_by_artistName(act) ### NEED TO HAVE CHECK STATEMENT HERE
        if id_return == 'NOT IN SPOTIFY':
            venue_genre = 'NONE'
        else:
            venue_genre = get_genre(id_return)
            #print("artist genre:", venue_genre)
            #list.append
            #for i in range(df.genre.size):
            
            mapped_genre = ''
            
            if any("funk" in s for s in venue_genre):
                mapped_genre = 0
        
            if any("rock" in s for s in venue_genre):
                mapped_genre = 1
                
            if any("punk" in s for s in venue_genre):
                mapped_genre = 1
        
            if any("reggae" in s for s in venue_genre):
                mapped_genre = 2
        
            if any("r&b" in s for s in venue_genre):
                mapped_genre = 3
        
            if any("rap" in s for s in venue_genre):
                mapped_genre = 4
        
            if any("pop" in s for s in venue_genre):
                mapped_genre = 5
        
            if any("jazz" in s for s in venue_genre):
                mapped_genre = 6
        
            if any("electronic" in s for s in venue_genre):
                mapped_genre = 7
        
            if any("indie" in s for s in venue_genre):
                mapped_genre = 8
        
            if any("country" in s for s in venue_genre):
                mapped_genre = 9
            
        #list_mapped_genre = set(mapped_genre)
        #mapped_genre_list = list(dict.fromkeys(mapped_genre))
        venue_artist_dict[venue] = [mapped_genre]
    
    # if it is in the dict, add the act to its 
    else:
        if act not in venue_artist_dict[venue]:
            venue_artist_dict[venue].append(mapped_genre)

for key in venue_artist_dict:
    no_duplicate_values = []
    for i in venue_artist_dict[key]:
        if i not in no_duplicate_values:
            no_duplicate_values.append(i)
    
    venue_artist_dict[key] = no_duplicate_values
    
    while ('' in venue_artist_dict[key]):
        venue_artist_dict[key].remove('')
    
print(venue_artist_dict)

import math
import numpy as np

#
# FUNCTION TO ADD ELEMENTS TO DICT
#
def append_value(dict_obj, key, value):
    if key in dict_obj:
        if not isinstance(dict_obj[key], tuple):
            dict_obj[key] = [dict_obj[key]]
        dict_obj[key].append(value)
    else:
        dict_obj[key] = value

#
# GENRE CHECK FUNCTION
# if artist genre identity is a match for venue genre, match (hard constraints)
#

def genre_check(d, genre_of):
    #print('in genre check function')
    genre_filtered_dictionary = {}

    # iterate through artist's given constraints O(n)
    for i in genre_of:
        for key, value in d.items():
            for j in value:
                if i == j:
                    append_value(genre_filtered_dictionary, key, value)
                else:
                    pass

    return genre_filtered_dictionary

#
# SIZE MATCH FUNCTION
# if artist genre identity is a match for venue size, match (loose constraints)
#

def size_match_percentage(d,size_of):
    perfect_match_dict={}
    for key,value in d.items():
        num=value
        difference=abs(num-size_of)
        per=difference/size_of
        percentage1=1-per
        if(np.allclose(percentage1,.55)):
            perfect_match_dict[key]="{0:.0f}%".format(percentage1*100)
            None
    return perfect_match_dict

#
# OUTPUT SECTION
# change this portion when looking at different bands
#
for i in range(0,len(df)):

    print("\n\n\nARTIST:", df.band_name[i])
    artist_genre = []
    for j in range(0,len(df.mapped_genre[i])):
        artist_genre.append(int(df.mapped_genre[i][j]))

    print("Artist Genre: ", artist_genre)
    artist_size = df.followers[i]
    print("Artist Size (based on followers): ", artist_size)

    ## NEED TO FIX THIS LINE 
    ##- issue with format of list as list of strings when should be a list of integers 
    #perfect = genre_check(venue_artist_dict,[1])
    perfect = genre_check(venue_artist_dict,artist_genre)
    print('perfect: ', perfect)

    final_return = size_match_percentage(perfect,artist_size)
    print(final_return)


{"The Howlin' Wolf": [0, 5], 'Unknown venue': [5, 8], 'Southern Rep Theatre': [5], 'The Fillmore': [7], "Tipitina's Uptown": [5, 8], 'Treme Art & Music Lounge': [0, 5], 'Destrehan Plantation': [], 'Music Box Village': [8], 'One Eyed Jacks': [5, 1], 'Twist of Lime': [], 'Ernest N Morial Convention Center': [], 'UNO Lakefront Arena': [1], 'Joy Theater': [9], 'Civic Theatre': [5, 1], 'House of Blues New Orleans presented by Cricket Wireless': [8, 1], 'Smoothie King Center': [1, 8], 'The Maison': [1], 'Fillmore New Orleans': [8, 4], 'Blue Nile': [8], 'Saenger Theatre New Orleans': [4]}



ARTIST: Turnstile
Artist Genre:  [1]
Artist Size (based on followers):  101949
perfect:  {'One Eyed Jacks': [5, 1], 'UNO Lakefront Arena': [1], 'Civic Theatre': [5, 1], 'House of Blues New Orleans presented by Cricket Wireless': [8, 1], 'Smoothie King Center': [1, 8], 'The Maison': [1]}
{}



ARTIST: NoMBe
Artist Genre:  [1, 5, 8]
Artist Size (based on followers):  148220
perfect:  {'One Eyed Jacks': [[5,